# Tutorial no. 2 of SpeechBrain-MOABB: Setting up hyper-parameter tuning

## **Prerequisites**


### Download SpeechBrain-MOABB

SpeechBrain-MOABB can be downloaded from the GitHub repository listed below.

In [ ]:
!git clone https://github.com/speechbrain/benchmarks

### Install SpeechBrain and SpeechBrain-MOABB requirements, and install SpeechBrain

In [ ]:
%%capture
!pip install speechbrain

%cd /content/benchmarks/benchmarks/MOABB
!pip install -r extra-requirements.txt # Install additional dependencies


## **Define the yaml file including the hyper-parameter search space**

Let us use the same yaml file as in the *Tutorial no. 1 of SpeechBrain-MOABB: Setting up EEG decoding*. However, in this case we assume that some hyper-parameters are not optimal. For example, we assume that the low and high cut-off frequencies for band-pass filtering should be optimized, together with the number of epochs, the learning rate, and few network hyper-parameters (e.g., the number of convolutional kernels and the kernel size of the first layer of EEGNet).

We provide a CLI for performing hyper-parameter search, by using the `./run_hparam_optimization.sh` script, that performs the hyper-parameter search iterations calling multiple times the `./run_experiments.sh` script. The script assumes that Orion flags are directly included in the specified YAML hyper-parameter file using comments. Thus, you can easily define the search space for each hyper-parameter by commenting:

```yaml
dropout: 0.1748  # @orion_step1: --dropout~"uniform(0.0, 0.5)"
```

In this case, dropout rate will be sampled using an uniform distribution between 0 and 0.5. See Orion documentation for the supported distributions.

`./run_hparam_optimization.sh` supports multi-step hyper-parameter optimization.
Briefly, you can optimize a subset of hyper-parameters while keeping the others fixed. After finding their optimal values, we utilize them as a foundation for optimizing another set of hyper-parameters. Furthermore, once hyper-parameter tuning is completed, the optimal decoding pipeline is re-trained and re-evaluated for N times for providing a robust evaluation of the performance, to reduce the variability of the performance due to different random initializations (seed variability), by setting the parameter `nruns_eval`. Besides the options provided by the `./run_experiments.sh` CLI, `./run_hparam_optimization.sh` introduces other options. For example, the user can change the amount of signals to use during hyper-parameter search, crucial for reducing computational time on large datasets, by setting the number of participants and sessions to use for hyper-parameter search (`nsbj_hpsearch`, `nsess_hpsearch`).

To optimize a hyper-parameter in a second step, follow this syntax in the YAML file:

```yaml
# cutcat (disabled when min_num_segments=max_num_segments=1)
max_num_segments: 6 # @orion_step2: --max_num_segments~"uniform(2, 6, discrete=True)"
```

For brevity, the number of epochs was tuned here only between 50 and 200 epochs.

In [ ]:
tuned_hyperparams = """
# DATASET HPARS
fmin: 1 # @orion_step1: --fmin~"uniform(0.1, 5, precision=2)"
fmax: 40 # @orion_step1: --fmax~"uniform(20.0, 50.0, precision=3)"

# TRAINING HPARS
number_of_epochs: 1000 # @orion_step1: --number_of_epochs~"uniform(50, 200, discrete=True)"
lr: 0.0001 # @orion_step1: --lr~"choices([0.01, 0.005, 0.001, 0.0005, 0.0001])"

# MODEL
cnn_temporal_kernels: 8 # @orion_step1: --cnn_temporal_kernels~"uniform(4, 64,discrete=True)"
cnn_temporal_kernelsize: 62 # @orion_step1: --cnn_temporal_kernelsize~"uniform(24, 62,discrete=True)"

"""

other_hyperparams = """
seed: 1234
__set_torchseed: !apply:torch.manual_seed [!ref <seed>]

# DIRECTORIES
data_folder: !PLACEHOLDER  #'/path/to/dataset'. The dataset will be automatically downloaded in this folder
cached_data_folder: !PLACEHOLDER #'path/to/pickled/dataset'
output_folder: !PLACEHOLDER #'path/to/results'

# DATASET HPARS
# Defining the MOABB dataset.
dataset: !new:moabb.datasets.BNCI2014001
save_prepared_dataset: True # set to True if you want to save the prepared dataset as a pkl file to load and use afterwards
data_iterator_name: 'leave-one-session-out'
target_subject_idx: 0
target_session_idx: 1
events_to_load: null # all events will be loaded
original_sample_rate: 250 # Original sampling rate provided by dataset authors
sample_rate: 125 # Target sampling rate (Hz)
# band-pass filtering cut-off frequencies
n_classes: 4
tmin: 0.
tmax: 4.0
# number of steps used when selecting adjacent channels from a seed channel (default at Cz)
n_steps_channel_selection: 3
T: !apply:math.ceil
    - !ref <sample_rate> * (<tmax> - <tmin>)
C: 22
test_with: 'last' # 'last' or 'best'
test_key: "acc" # Possible opts: "loss", "f1", "auc", "acc"

# METRICS
f1: !name:sklearn.metrics.f1_score
    average: 'macro'
acc: !name:sklearn.metrics.balanced_accuracy_score
cm: !name:sklearn.metrics.confusion_matrix
metrics:
    f1: !ref <f1>
    acc: !ref <acc>
    cm: !ref <cm>

# TRAINING HPARS
n_train_examples: 100  # it will be replaced in the train script
# checkpoints to average
avg_models: 10
# Learning rate scheduling (cyclic learning rate is used here)
max_lr: !ref <lr> # Upper bound of the cycle (max value of the lr)
base_lr: 0.00000001 # Lower bound in the cycle (min value of the lr)
step_size_multiplier: 5 #from 2 to 8
step_size: !apply:round
    - !ref <step_size_multiplier> * <n_train_examples> / <batch_size>
lr_annealing: !new:speechbrain.nnet.schedulers.CyclicLRScheduler
    base_lr: !ref <base_lr>
    max_lr: !ref <max_lr>
    step_size: !ref <step_size>
label_smoothing: 0.0
loss: !name:speechbrain.nnet.losses.nll_loss
    label_smoothing: !ref <label_smoothing>
optimizer: !name:torch.optim.Adam
    lr: !ref <lr>
epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter  # epoch counter
    limit: !ref <number_of_epochs>
batch_size: 32
valid_ratio: 0.2

# DATA NORMALIZATION
dims_to_normalize: 1 # 1 (time) or 2 (EEG channels)
normalize: !name:speechbrain.processing.signal_processing.mean_std_norm
    dims: !ref <dims_to_normalize>

# MODEL
input_shape: [null, !ref <T>, !ref <C>, null]
cnn_spatial_depth_multiplier: 2
cnn_spatial_max_norm: 1.
cnn_spatial_pool: 4
cnn_septemporal_depth_multiplier: 1
cnn_septemporal_point_kernels: !ref <cnn_temporal_kernels> * <cnn_spatial_depth_multiplier> * <cnn_septemporal_depth_multiplier>
cnn_septemporal_kernelsize: 16
cnn_septemporal_pool: 8
cnn_pool_type: 'avg'
dense_max_norm: 0.25
dropout: 0.5
activation_type: 'elu'

model: !new:models.EEGNet.EEGNet
    input_shape: !ref <input_shape>
    cnn_temporal_kernels: !ref <cnn_temporal_kernels>
    cnn_temporal_kernelsize: [!ref <cnn_temporal_kernelsize>, 1]
    cnn_spatial_depth_multiplier: !ref <cnn_spatial_depth_multiplier>
    cnn_spatial_max_norm: !ref <cnn_spatial_max_norm>
    cnn_spatial_pool: [!ref <cnn_spatial_pool>, 1]
    cnn_septemporal_depth_multiplier: !ref <cnn_septemporal_depth_multiplier>
    cnn_septemporal_point_kernels: !ref <cnn_septemporal_point_kernels>
    cnn_septemporal_kernelsize: [!ref <cnn_septemporal_kernelsize>, 1]
    cnn_septemporal_pool: [!ref <cnn_septemporal_pool>, 1]
    cnn_pool_type: !ref <cnn_pool_type>
    activation_type: !ref <activation_type>
    dense_max_norm: !ref <dense_max_norm>
    dropout: !ref <dropout>
    dense_n_neurons: !ref <n_classes>

"""

sample_hyperparams = tuned_hyperparams + other_hyperparams

In [ ]:
# Save the yaml file on disk
f = open('/content/sample_hyperparams.yaml', "w")
f.write(sample_hyperparams)
f.close()

In [ ]:
!./run_hparam_optimization.sh --hparams '/content/sample_hyperparams.yaml' \
--data_folder '/content/data/BNCI2014001'\
--cached_data_folder '/content/data' \
--output_folder '/content/results/hyperparameter-search/BNCI2014001' \
--nsbj 9 --nsess 2 --nruns 1 --train_mode 'leave-one-session-out' \
--exp_name 'hyperparameter-search' \
--nsbj_hpsearch 1 --nsess_hpsearch 1 \
--nruns_eval 1 \
--eval_metric acc \
--exp_max_trials 50